In [ ]:
from getpass import getpass
from dl import authClient as ac, queryClient as qc
import pandas as pd
import seaborn as sns
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
sns.set_theme(style="whitegrid")

In [2]:
# defn columns to get by group
ids = "m.id1 as ls_id, m.id2 as sdss_specobjid, pp.objid as sdss_bestobjid, ls.type" 
ras_decs = "ls.ra as ls_ra, so.ra as so_ra, pp.ra as pp_ra, ls.dec as la_dec, so.dec as so_dec, pp.dec as pp_dec"
ls_mags = "ls.dered_mag_g as ls_dered_g, ls.dered_mag_r as ls_dered_r, ls.dered_mag_w1 as ls_dered_w1, ls.dered_mag_w2 as ls_dered_w2, ls.dered_mag_w3 as ls_dered_w3, ls.dered_mag_w4 as ls_dered_w4, ls.dered_mag_z as ls_dered_z"
sdss_mags = "pp.dered_g as sdss_dered_g, pp.dered_i as sdss_dered_i, pp.dered_r as sdss_dered_r, pp.dered_u as sdss_dered_u, pp.dered_z as sdss_dered_z"
# Change this name? Its technically not r-band, though it is said to be same in every band so...?? 
radii = "ls.shape_r as ls_radius_r, pp.petror50_r as sdss_radius_r_petror50, pp.devrad_r as sdss_radius_r_devaucouleurs, pp.exprad_r as sdss_radius_r_exprad" 
shapes = "ls.shape_e1 as ls_ellipticity_component_1, ls.shape_e2 as ls_ellipticity_component_2"
sersic = "ls.sersic"
chi_squareds = "ls.dchisq_3 as ls_dchisq_dev, ls.dchisq_4 as ls_dchisq_exp, ls.rchisq_r as ls_rchisq_r"


columns = ids + ", " + ras_decs + ", " + ls_mags + ", "  + sdss_mags + ", "  + shapes + ", "  + sersic + ", "  + radii + ", " + chi_squareds

query = "\
   SELECT " + columns + " \
     FROM ls_dr9.x1p5__tractor__sdss_dr17__specobj m \
LEFT JOIN ls_dr9.tractor ls       ON ls.ls_id = m.id1 \
LEFT JOIN sdss_dr17.specobj so    ON so.specobjid = m.id2 \
LEFT JOIN sdss_dr17.photoplate pp ON pp.objid = so.bestobjid \
    WHERE ls.shape_r > 0 \
      AND pp.objid IS NOT NULL"
result = qc.query(sql=query,fmt='pandas')
print("Types: {}".format(result.type.unique()))

In [ ]:
result.to_csv('ls_sdss_data.csv')

Or just keep the offline data

In [ ]:
result = pd.read_csv('ls_sdss_data.csv')

In [ ]:
df_sdss_ls = result[["ls_id", "sdss_specobjid", "type", "ls_dered_g", "sdss_dered_g", "ls_dered_r", "sdss_dered_r", "ls_dered_z", "sdss_dered_z", "ls_radius_r", "sdss_radius_r_devaucouleurs", "sdss_radius_r_exprad", "sdss_radius_r_petror50", "ls_ellipticity_component_1", "ls_ellipticity_component_2", "ls_dchisq_exp", "ls_rchisq_r", "sersic"]].copy()
df_sdss_ls

## Read Meert data (for SDSS)

In [ ]:
from astropy.io import fits

In [ ]:
with fits.open('/home/abs/Research/Meert_Catalog/meert_et_al_data_tables_v2/UPenn_PhotDec_Models_rband.fits') as data:
    meert_data_models = pd.DataFrame(np.array(data[5].data).byteswap().newbyteorder()) # .fits are all bigendian, pandas / scipy littleendian

with fits.open('/home/abs/Research/Meert_Catalog/meert_et_al_data_tables_v2/UPenn_PhotDec_CAST.fits') as data:
    meert_cast = pd.DataFrame(np.array(data[1].data).byteswap().newbyteorder())

In [ ]:
meert_data = meert_data_models.join(meert_cast, how="left")
meert_data

In [ ]:
df = df_sdss_ls.set_index('sdss_specobjid').join(meert_data.set_index('specobjid'), how="outer")
df

In [ ]:
specobjs_ls_sdss = df_sdss_ls['sdss_specobjid'].unique()
specobjs_meert = meert_data['specobjid'].unique()
# shared_specobjs = set(specobjs_ls_sdss).intersection(specobjs_meert)
# display(shared_specobjs)
display(specobjs_ls_sdss)
display(specobjs_meert)

## Generate new columns
- Ellipticity (b/a) https://www.legacysurvey.org/dr9/catalogs/#ellipticities
- Circularized radius of LS semimajor axes
- Differences in radius measurements (+ percent diff)
- Differences in r-band magnitudes (+ percent diff)

In [ ]:
# Remove plain duplicates 
print("pre-cut: {}".format(df.shape[0]))
df["is_ls_duplicate"] = df.duplicated(subset=['ls_id'], keep=False)
df["is_sdss_duplicate"] = df.duplicated(subset=['sdss_specobjid'], keep=False)
print("post-cut: {}".format(df.shape[0]))

In [ ]:
# Add more radial comparison parameters
df["ls_e"] = np.hypot(df["ls_ellipticity_component_1"], df["ls_ellipticity_component_2"])
df["ls_b_over_a"] = (1 - df["ls_e"]) / (1 + df["ls_e"])

# circularization of ls "radius_r"
df["ls_radius_r_circularized"] = df["ls_radius_r"]*np.sqrt(df["ls_b_over_a"]) # equivalent area radius for given semimajor axis and b/a (r = sqrt(a*b))

# magnitude differences
df["dered_mag_diff"] = df["ls_dered_r"] - df["sdss_dered_r"]
df["dered_percent_diff"] = df["dered_mag_diff"] / df["ls_dered_r"] # again % diff from ls data
df["abs_dered_mag_diff"] = np.abs(df["dered_mag_diff"])

## Scrub Data
- Filter out duplicate-match rows (choose closest r-band magnitude)
- Remove data without petrorsian radius measurements (in SDSS this is given by "-9999.0")
- Remove data without r-band magnitudes in either survey (in LS this is "inf")

#### Remove Duplicates

In [ ]:
df.sort_values(['ls_id', 'abs_dered_mag_diff'], ascending=[True, False])
df = df.drop_duplicates(subset=['sdss_specobjid'], keep='first')
print("Length: {}".format(df.shape[0]))

In [ ]:
# just checking that we dropped the correct ones
df[df["ls_id"] == 9907740037940626]

In [ ]:
# make sure no duplicates left (should show empty DF)
df_check = df.copy()
df_check["is_sdss_still_duplicated"] = df_check.duplicated(subset=['sdss_specobjid'], keep=False)
df_check[df_check["is_sdss_still_duplicated"] == True]

In [ ]:
# no duplicate LS ids :)
df_view_ls_dups = df[df["is_ls_duplicate"] == True]
df_view_ls_dups

#### Remove no petror_r

In [ ]:
# has petrosian half-light radius
df = df[df["sdss_radius_r_petror50"] > -9000]
print("Length: {}".format(df.shape[0]))

#### Remove no r-band magnitude

In [ ]:
df = df[~df["ls_dered_r"].isin([np.inf])]
print("Length: {}".format(df.shape[0]))

## Science Filters
- Magnitudes brighter than 21
- Type (everything except PSF, DUP)

In [ ]:
df = df[df["ls_dered_r"] <= 21]
# PSF and DUP already not in joined dataset
# df = df[~df["type"].isin(["PSF", "DUP"])]
print("Length: {}".format(df.shape[0]))

In [ ]:
data_no_rex = df[df["type"] != "REX"].copy()

## Compare subsets

In [ ]:
# Sample type in ["DEV", "EXP", "REX", "any"]
# sdss_radius_type in "petror" or "sample" (e.g. "type" for sample DEV would choose sdss_radius_r_devaucouleurs"
# choose circ=True to use circularized radius
def compare_radii(sample_type="DEV", sdss_radius_type="petror", circ=False):
    
    # data set
    if sample_type == "any":
        df_type = df.copy() 
    else: df_type = df[df["type"] == sample_type].copy()
    
    # sdss radius to use  (replace prints with try-catches later)
    if sdss_radius_type == "petror":
        sdss_radius = "sdss_radius_r_petror50"
    elif sdss_radius_type == "sample":
        if sample_type == "DEV":
            sdss_radius = "sdss_radius_r_devaucouleurs"
        elif sample_type == "EXP":
            sdss_radius = "sdss_radius_r_exprad"
        else:
            sdss_radius = "sdss_radius_r_petror50"
            print("type {} has no special radius - defaults petror50 radius".format(sample_type))
    else: 
        print("Invalid radius type. Sample types in ['DEV', 'EXP', 'REX', 'SER', 'any'] sdss_radius_type in ['petror', 'sample']")
    
    # ls radius to use
    ls_radius = "ls_radius_r_circularized" if circ else "ls_radius_r"
        
    # Do it to it
    df_type["radius_diff"] = df_type[sdss_radius] - df_type[ls_radius]
    df_type["radius_diff_percent"] = 100 * df_type["radius_diff"] / df_type[ls_radius]
    # df_type["sdss_radius_type"] = sdss_radius.split("sdss_radius_r_")[1]
    df_type["sdss_radius_type"] = sdss_radius_type
    df_type["circularized"] = "Yes" if circ else "No"
    df_type = df_type[["type", "sdss_radius_type", "circularized", "ls_b_over_a", "radius_diff", "radius_diff_percent", "sersic"]].copy()
    
    return df_type

In [ ]:
plot_data = pd.DataFrame()

for type in ['DEV', 'EXP', 'SER']:
    for sdss_radius_type in ['petror', 'sample']:
        for circ in [True, False]:
            df_type = compare_radii(type, sdss_radius_type, circ)
            plot_data = plot_data.append(df_type)

In [ ]:
plot_data["column"] = "$r_{SDSS}:$" + plot_data["sdss_radius_type"] + ", " + "$r_{ls}$ circlrzd: " + plot_data["circularized"]

In [ ]:
def hist2d(x, y, **kwargs):
    kwargs.pop("color", None)
    plt.hist2d(x, y, bins=(100,10000), **kwargs)
    plt.ylim(-100,100)
    plt.xlim(0,1)

g = sns.FacetGrid(plot_data, row="type", col="column", margin_titles=True)
g.map(hist2d, "ls_b_over_a", "radius_diff_percent", cmap='viridis')
g.set_titles(col_template="{col_name}", row_template="{row_name}")

## Sersic sub-plots

In [ ]:
sersic_vals = np.arange(0, plot_data['sersic'].max()+0.5, 1)
bin_labels = []
for i in range (0, len(sersic_vals)-1):
    bin_labels.append(str(sersic_vals[i]) + "-" + str(sersic_vals[i+1]))

In [ ]:
sersic_data = compare_radii(sample_type="SER", sdss_radius_type="petror", circ=False)
sersic_data_circ = compare_radii(sample_type="SER", sdss_radius_type="petror", circ=True)

sersic_data

In [ ]:
sersic_data['sersic_group'] = (np.select([sersic_data['sersic'].between(i, j, inclusive='right') 
                           for i,j in zip(sersic_vals, sersic_vals[1:])], 
                          bin_labels))
sersic_data_circ['sersic_group'] = (np.select([sersic_data_circ['sersic'].between(i, j, inclusive='right') 
                           for i,j in zip(sersic_vals, sersic_vals[1:])], 
                          bin_labels))
sersic_data = sersic_data.sort_values('sersic_group')
sersic_data_circ = sersic_data_circ.sort_values('sersic_group')

In [ ]:
sersic_data_all = sersic_data_circ.append(sersic_data)


sersic_vals = np.arange(0, plot_data['sersic'].max()+0.5, 2)
bin_labels = []
for i in range (0, len(sersic_vals)-1):
    bin_labels.append(str(sersic_vals[i]) + "-" + str(sersic_vals[i+1]))

sersic_data_all['sersic_group'] = (np.select([sersic_data_all['sersic'].between(i, j, inclusive='right') 
                           for i,j in zip(sersic_vals, sersic_vals[1:])], 
                          bin_labels))

sersic_data_all = sersic_data_all.sort_values('sersic_group')

In [ ]:
# https://www.sdss4.org/dr12/algorithms/magnitudes/#mag_petro

def hist2d(x, y, **kwargs):
    kwargs.pop("color", None)
    plt.hist2d(x, y, bins=(100,10000), **kwargs)
    plt.ylim(-100,100)
    plt.xlim(0,1)

g = sns.FacetGrid(sersic_data_all, row="circularized", col="sersic_group", margin_titles=True, gridspec_kws={"wspace":0, "hspace":0})
g.map(hist2d, "ls_b_over_a", "radius_diff_percent", cmap='viridis')
g.set_titles(col_template="Sersic index: {col_name}", row_template="LS r circularized? {row_name}")
g.set_xlabels("$\longleftarrow$ line (b/a) circle $\longrightarrow$")
g.set_ylabels('')
g.fig.supylabel("% ($r_{SDSS petro}$  - $r_{LS}$)")

In [ ]:
def hist2d(x, y, **kwargs):
    kwargs.pop("color", None)
    plt.hist2d(x, y, bins=(100,10000), **kwargs)
    plt.ylim(-100,100)
    plt.xlim(0,1)

g = sns.FacetGrid(sersic_data, col="sersic_group", margin_titles=True, col_wrap=3)
g.map(hist2d, "ls_b_over_a", "radius_diff_percent", cmap='viridis')
g.set_titles(col_template="{col_name}")
g.set_xlabels("")
g.set_ylabels('')
g.fig.supylabel("% ($r_{SDSS petro}$  - $r_{LS}$)")
g.fig.supxlabel("$\longleftarrow$ line             (b/a)              circle $\longrightarrow$")

In [ ]:
def hist2d(x, y, **kwargs):
    kwargs.pop("color", None)
    plt.hist2d(x, y, bins=(100,10000), **kwargs)
    plt.ylim(-100,100)
    plt.xlim(0,1)

g = sns.FacetGrid(sersic_data_circ, col="sersic_group", margin_titles=True, col_wrap=3)
g.map(hist2d, "ls_b_over_a", "radius_diff_percent", cmap='viridis')
g.set_titles(col_template="{col_name}")
g.set_xlabels("")
g.set_ylabels('')
g.fig.supylabel("% ($r_{SDSS petro}$  - $r_{LS}$ CIRCULARIZED)")
g.fig.supxlabel("$\longleftarrow$ line             (b/a)              circle $\longrightarrow$")